# Diffusion（扩散）— 物理/概念模型演示

本笔记本演示如何将“邻接矩阵的幂”理解为网络中的扩散（传播）过程：在源节点滴下一滴“墨水”，它会沿着边逐步扩散，t=k 时的分布由 \(A^k\)（或归一化后的 \(P^k\)）决定。

- **t=0**：墨水只在源节点 i。
- **t=1**：沿着 i 的所有出边流向其直接邻居（由邻接矩阵 \(A\) 的第 i 行描述）。
- **t=2**：继续从这些邻居扩散到它们的邻居（由 \(A^2\) 描述）。
- **t=k**：网络中的分布由 \(A^k\)（或 \(P^k\)）描述，\((A^k)_{ij}\) 代表从 i 到 j 的通路数量（或影响力强度）。

本演示支持：
- **矩阵规模 N**（节点数）
- **传播步数 K**（扩散尺度）
- **邻接构造**：顺序链式连接 + 少量随机连接
- **是否行归一化**：使用转移矩阵 \(P\)（按行归一）来模拟概率扩散
- **是否有向**：可切换有向/无向
- **源节点**与**随机种子**可调

交互区会展示：
- 网络图（按节点顺序排布），节点颜色/大小表示在 t=K 时的“墨水”强度
- t=K 时从源点出发的分布柱状图
- 矩阵幂 \(A^k\) 或 \(P^k\) 的热力图


In [1]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from dataclasses import dataclass
import networkx as nx
from typing import Tuple, Optional

np.set_printoptions(precision=3, suppress=True)

@dataclass
class AdjacencyConfig:
    num_nodes: int = 50
    num_random_edges: int = 50
    directed: bool = False
    allow_self_loops: bool = False
    random_weight: bool = False
    symmetric_weights: bool = True
    min_weight: float = 0.0
    max_weight: float = 1.0
    seed: Optional[int] = 42


def build_chain_plus_random_edges(cfg: AdjacencyConfig) -> sp.csr_matrix:
    """Construct adjacency with a chain backbone plus a small number of random edges.

    - Chain edges: (0->1->2->...->N-1). If undirected, add symmetric edges.
    - Random edges: sample distinct pairs (u, v), avoid duplicates and (optionally) self-loops.
    - Optionally weighted: if random_weight=True, weights ~ Uniform[0,1]; else weight=1.
    - If symmetric_weights=True, symmetrize final weights by (A + A^T)/2.
    """
    rng = np.random.default_rng(cfg.seed)
    n = cfg.num_nodes
    rows = []
    cols = []
    data = []

    def add_edge(u: int, v: int, w: float = 1.0):
        rows.append(u)
        cols.append(v)
        data.append(w)

    def sample_weight() -> float:
        return float(rng.random())

    # Chain backbone
    for u in range(n - 1):
        v = u + 1
        w = 1.0  # chain edges fixed weight
        add_edge(u, v, w)
        if not cfg.directed:
            add_edge(v, u, w)

    # Random edges
    existing = set(zip(rows, cols))
    attempts = 0
    target = cfg.num_random_edges
    max_attempts = max(10 * target, 1000)
    while len(existing) < len(rows):
        # keep set consistent with rows/cols
        existing = set(zip(rows, cols))
        break
    while target > 0 and attempts < max_attempts:
        u = int(rng.integers(0, n))
        v = int(rng.integers(0, n))
        if not cfg.allow_self_loops and u == v:
            attempts += 1
            continue
        if (u, v) in existing:
            attempts += 1
            continue
        w = sample_weight()
        add_edge(u, v, w)
        existing.add((u, v))
        if not cfg.directed and (v, u) not in existing:
            add_edge(v, u, w if cfg.symmetric_weights else sample_weight())
            existing.add((v, u))
        target -= 1
        attempts += 1

    A = sp.csr_matrix((data, (rows, cols)), shape=(n, n), dtype=float)
    # Remove potential duplicate edges by summing
    A.sum_duplicates()

    if cfg.symmetric_weights:
        # Enforce numeric symmetry of weights
        A = (A + A.T) * 0.5
        A = A.tocsr()
        A.sum_duplicates()

    return A


def adjacency_to_networkx(A: sp.csr_matrix, directed: bool) -> nx.Graph:
    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    n = A.shape[0]
    G.add_nodes_from(range(n))
    coo = A.tocoo()
    for u, v, w in zip(coo.row, coo.col, coo.data):
        G.add_edge(int(u), int(v), weight=float(w))
    return G



In [2]:
def row_normalize(A: sp.csr_matrix) -> sp.csr_matrix:
    """Row-normalize sparse matrix: P = D^{-1} A, with zero-row handling."""
    row_sums = np.asarray(A.sum(axis=1)).ravel()
    inv = np.zeros_like(row_sums)
    nonzero = row_sums > 0
    inv[nonzero] = 1.0 / row_sums[nonzero]
    D_inv = sp.diags(inv)
    return D_inv @ A


def symmetric_normalize(A: sp.csr_matrix) -> sp.csr_matrix:
    """Symmetric normalization for undirected graphs: \tilde P = D^{-1/2} A D^{-1/2}."""
    deg = np.asarray(A.sum(axis=1)).ravel()
    inv_sqrt = np.zeros_like(deg)
    nonzero = deg > 0
    inv_sqrt[nonzero] = 1.0 / np.sqrt(deg[nonzero])
    D_is = sp.diags(inv_sqrt)
    return D_is @ A @ D_is


def power_sparse(A: sp.csr_matrix, k: int) -> sp.csr_matrix:
    """Compute A^k efficiently with exponentiation by squaring (sparse)."""
    if k < 0:
        raise ValueError("k must be >= 0")
    if k == 0:
        return sp.identity(A.shape[0], format='csr', dtype=A.dtype)
    if k == 1:
        return A.tocsr()
    # exponentiation by squaring
    result = sp.identity(A.shape[0], format='csr', dtype=A.dtype)
    base = A.tocsr()
    exp = k
    while exp > 0:
        if exp % 2 == 1:
            result = result @ base
        base = base @ base
        exp //= 2
    return result


def diffusion_distribution(A: sp.csr_matrix, source: int, k: int, use_row_normalized: bool, directed: bool) -> np.ndarray:
    """Return distribution vector after k steps from a one-hot source.

    - If use_row_normalized=True:
        - directed=True: use P = D^{-1}A (row-normalized random walk)
        - directed=False: use \tilde P = D^{-1/2}AD^{-1/2} (symmetric normalization)
    - Otherwise use raw adjacency power A^k (counts of k-step walks).
    """
    n = A.shape[0]
    if source < 0 or source >= n:
        raise ValueError("source out of range")
    if use_row_normalized:
        M = row_normalize(A) if directed else symmetric_normalize(A)
    else:
        M = A
    Mk = power_sparse(M, k)
    e = np.zeros(n, dtype=float)
    e[source] = 1.0
    dist = e @ Mk  # shape (n,)
    return np.asarray(dist).ravel()



In [6]:
# If running outside JupyterLab, uncomment:
# %matplotlib inline

from ipywidgets import interact, IntSlider, Checkbox, Dropdown, HBox, VBox, Layout, FloatSlider, Button
import ipywidgets as widgets


def plot_diffusion(A: sp.csr_matrix, source: int, k: int, use_row_normalized: bool, directed: bool):
    n = A.shape[0]
    # Compute distribution
    dist = diffusion_distribution(A, source=source, k=k, use_row_normalized=use_row_normalized, directed=directed)

    # Graph
    G = adjacency_to_networkx(A, directed=directed)
    pos = {i: (i, 0.0) for i in range(n)}  # 1D layout by index

    vmax = max(dist.max(), 1e-9)
    norm = Normalize(vmin=0.0, vmax=vmax)
    node_colors = [plt.cm.viridis(norm(dist[i])) for i in range(n)]
    node_sizes = 200 * (1 + 2 * (dist / (vmax + 1e-12)))

    fig = plt.figure(figsize=(15, 4))

    # Subplot 1: Graph view
    ax1 = fig.add_subplot(1, 3, 1)
    nx.draw(
        G,
        pos,
        with_labels=True,
        node_color=node_colors,
        node_size=node_sizes,
        edge_color="#999999",
        width=1.0,
        arrows=directed,
        ax=ax1,
    )
    ax1.set_title(f"Graph (n={n}), source={source}, k={k}")

    # Subplot 2: Distribution bar
    ax2 = fig.add_subplot(1, 3, 2)
    ax2.bar(range(n), dist, color=plt.cm.viridis(norm(dist)))
    ax2.set_title("Distribution at t = K")
    ax2.set_xlabel("Node index")
    ax2.set_ylabel("Intensity" if not use_row_normalized else "Probability")

    # Subplot 3: Matrix power heatmap
    ax3 = fig.add_subplot(1, 3, 3)
    if use_row_normalized:
        M = row_normalize(A) if directed else symmetric_normalize(A)
    else:
        M = A
    Mk = power_sparse(M, k).todense()
    im = ax3.imshow(Mk, cmap="magma")
    fig.colorbar(im, ax=ax3, fraction=0.046, pad=0.04)
    title_op = "P^k" if (use_row_normalized and directed) else ("~P^k" if use_row_normalized else "A^k")
    ax3.set_title(title_op + " heatmap")
    ax3.set_xlabel("j")
    ax3.set_ylabel("i")

    plt.tight_layout()
    plt.show()


controls_layout = Layout(width='300px')

@interact(
    N=IntSlider(value=40, min=5, max=300, step=1, description='N', layout=controls_layout),
    K=IntSlider(value=5, min=0, max=40, step=1, description='K', layout=controls_layout),
    source=IntSlider(value=0, min=0, max=199, step=1, description='source', layout=controls_layout),
    random_edges=IntSlider(value=40, min=0, max=400, step=1, description='rand edges', layout=controls_layout),
    directed=Checkbox(value=False, description='directed'),
    normalize=Checkbox(value=True, description='row-normalize'),
    random_weight=Checkbox(value=False, description='weighted[0,1]'),
    symmetric_weights=Checkbox(value=True, description='symmetric weights'),
    seed=IntSlider(value=42, min=0, max=10000, step=1, description='seed', layout=controls_layout),
)
def interactive_diffusion(N, K, source, random_edges, directed, normalize, random_weight, symmetric_weights, seed):
    source = int(np.clip(source, 0, max(0, N-1)))
    cfg = AdjacencyConfig(
        num_nodes=int(N),
        num_random_edges=int(random_edges),
        directed=bool(directed),
        allow_self_loops=False,
        random_weight=bool(random_weight),
        symmetric_weights=bool(symmetric_weights),
        min_weight=0.0,
        max_weight=1.0,
        seed=int(seed),
    )
    A = build_chain_plus_random_edges(cfg)
    plot_diffusion(A, source=source, k=int(K), use_row_normalized=bool(normalize), directed=bool(directed))



interactive(children=(IntSlider(value=40, description='N', layout=Layout(width='300px'), max=300, min=5), IntS…

## 使用说明与说明

- 在交互控件中调整：
  - **N**：节点数（矩阵规模）。
  - **K**：传播步数（扩散尺度），越大代表扩散时间越长。
  - **source**：源节点索引，在 t=0 时只有该节点有“墨水”。
  - **rand edges**：在链式骨架之上增加的随机边数量。
  - **directed**：是否有向；如果关闭，则为无向图（对称）。
  - **row-normalize**：
    - 有向图：使用 \(P=D^{-1}A\) 进行随机游走（每行和为 1）。
    - 无向图：使用对称归一 \(\tilde P = D^{-1/2}AD^{-1/2}\)，保持算子与其幂 \(\tilde P^k\) 的对称性。
  - **weighted**：随机边权重在 [0, 1] 区间均匀采样；关闭时所有边权均为 1（链式骨架边恒为 1）。
  - **seed**：随机数种子，保证可复现实验。

- 图示：
  - 左图为图结构的可视化，节点颜色/大小反映 t=K 时从源点出发到达该节点的强度（或概率）。
  - 中图为 t=K 分布的柱状图，横轴为节点，纵轴为强度/概率。
  - 右图为 \(A^k\) 或 \(P^k\)/\(\tilde P^k\) 的热力图。第 i 行第 j 列即相应矩阵的 \((i,j)\)。

- 物理/概念解释：
  - t=1：沿邻接边传播一次；
  - t=2：从第一层邻居继续传播；
  - t=k：总的 k 步可达路径的强度之和。当多条路径通向同一节点时，强度叠加；若使用概率扩散（归一化），则对应到达该节点的概率质量或归一化强度。

- 性能提示：
  - 适中规模下（N≤200，K≤20）都能流畅运行；更大规模请酌情减少 K 或关闭热力图。
